In [ ]:
import json
import os
from collections import defaultdict

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
import numpy as np

try:
    from peft import PeftModel
    HAVE_PEFT = True
except ImportError:
    HAVE_PEFT = False

# ======================
# CONFIG
# ======================
NEURON_JSON_PATH = "./data/movie_sgcate/neurons_by_genre.json"
TEST_JSON_PATH   = "./data/movie_sgcate/genre_test_all_genres_hard.json"

BASE_MODEL = "meta-llama/Meta-Llama-3-8B-Instruct"
USE_LORA   = False   # set True if you want to load your DLCRec LoRA
LORA_PATH  = "./model/movie_sgcate/BERT_GP_vlGP_ep25_0.001/"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MAX_SAMPLES_PER_GENRE = 32   # use a subset per genre to save time
AMPLIFY_SCALE         = 1.5  # factor to amplify neurons

# ======================
# 1. Load neurons
# ======================
if not os.path.exists(NEURON_JSON_PATH):
    raise FileNotFoundError(f"Neuron file not found: {NEURON_JSON_PATH}")

with open(NEURON_JSON_PATH, "r") as f:
    neurons_by_genre = json.load(f)

# convert to sets of (layer, idx)
N = {g: set(tuple(x) for x in v) for g, v in neurons_by_genre.items()}
genres = list(N.keys())
print(f"Loaded neurons for {len(genres)} genres.")

# ======================
# 2. Load test data
# ======================
if not os.path.exists(TEST_JSON_PATH):
    raise FileNotFoundError(f"Test file not found: {TEST_JSON_PATH}")

with open(TEST_JSON_PATH, "r") as f:
    raw = json.load(f)

test_samples = []
for ex in raw:
    text  = ex.get("text", None)
    genre = ex.get("genre", None)
    if isinstance(text, str) and isinstance(genre, str):
        test_samples.append({"text": text, "genre": genre})

print(f"Loaded {len(test_samples)} test samples.")

samples_by_genre = defaultdict(list)
for ex in test_samples:
    samples_by_genre[ex["genre"]].append(ex)

print("Genres in test data:", list(samples_by_genre.keys()))

# restrict to only genres that appear in both neuron file and test file
genres_eval = [g for g in genres if g in samples_by_genre]
print("Evaluating on genres:", genres_eval)


# ======================
# 3. Load model
# ======================
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    device_map="auto" if DEVICE == "cuda" else None,
)
if USE_LORA:
    if not HAVE_PEFT:
        raise ImportError("peft not installed but USE_LORA=True")
    model = PeftModel.from_pretrained(
        model,
        LORA_PATH,
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    )

model.eval()
print("Model loaded.")


# ======================
# 4. Hook: ablate / amplify neurons (safe)
# ======================
def register_neuron_hook(model, layer_id, neuron_ids, mode="ablate", scale=1.5):
    """
    mode: 'ablate' -> set activations to 0
          'amplify' -> multiply activations by scale
    neuron_ids: list of indices along hidden_dim (some may be invalid > hidden size)
    """
    target_module = model.model.layers[layer_id]

    def hook_fn(module, inputs, output):
        # output can be tensor or tuple
        if isinstance(output, tuple):
            hidden = output[0]
        else:
            hidden = output

        hidden_dim = hidden.size(-1)
        valid_ids = [i for i in neuron_ids if 0 <= i < hidden_dim]
        if not valid_ids:
            return output

        idx = torch.tensor(valid_ids, device=hidden.device)

        if mode == "ablate":
            hidden[..., idx] = 0
        elif mode == "amplify":
            hidden[..., idx] *= scale

        if isinstance(output, tuple):
            return (hidden,) + output[1:]
        else:
            return hidden

    handle = target_module.register_forward_hook(hook_fn)
    return handle


# ======================
# 5. Multiple-choice genre classification
# ======================
def compute_genre_logprob(model, prompt: str, genre_label: str):
    """
    Treat genre classification as multiple choice:
    We compute log P(genre_label | prompt) by running model on (prompt + genre_label)
    and summing token logprobs of the label tokens.
    """
    # Prompt + label
    full_text = prompt + genre_label
    enc = tokenizer(full_text, return_tensors="pt").to(model.device)
    input_ids = enc["input_ids"][0]

    # We want logprob of label tokens given prompt tokens
    # Split into: [prompt_ids | label_ids]
    prompt_ids = tokenizer(prompt, return_tensors="pt").to(model.device)["input_ids"][0]
    len_prompt = prompt_ids.size(0)
    len_full   = input_ids.size(0)
    label_ids  = input_ids[len_prompt:]

    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0))
        logits  = outputs.logits[0]  # [seq_len, vocab]

    logprobs = torch.log_softmax(logits, dim=-1)

    # sum logprob of label tokens at their positions
    total_logprob = 0.0
    for i, tok_id in enumerate(label_ids):
        pos = len_prompt + i - 1  # predict token at position using previous
        if pos < 0 or pos >= logprobs.size(0):
            continue
        total_logprob += logprobs[pos, tok_id].item()

    return total_logprob


def evaluate_genre_mode(genre_name: str, mode: str = "base", scale: float = 1.5):
    """
    Evaluate per-genre accuracy in given mode (base/ablate/amplify).
    We consider all candidate genres in genres_eval, pick argmax logprob.
    """
    samples = samples_by_genre[genre_name][:MAX_SAMPLES_PER_GENRE]
    if not samples:
        print(f"No samples for {genre_name}")
        return None

    # build neuron map for this genre
    neurons_g = N.get(genre_name, set())
    layer_to_ids = defaultdict(list)
    for layer, idx in neurons_g:
        layer_to_ids[layer].append(idx)

    # install hooks if not base
    handles = []
    if mode != "base":
        for layer_id, neuron_ids in layer_to_ids.items():
            handles.append(
                register_neuron_hook(
                    model, layer_id, neuron_ids,
                    mode="ablate" if mode == "ablate" else "amplify",
                    scale=scale
                )
            )

    correct = 0
    total   = 0

    for ex in samples:
        text  = ex["text"]
        true_g = ex["genre"]

        # build prompt that does NOT reveal the correct genre
        prompt = (
            "Read the following movie description and choose the most appropriate genre "
            "from the candidate list.\n\n"
            f"Description: {text}\n\n"
            f"Candidate genres: {', '.join(genres_eval)}\n"
            "Answer with only one genre name: "
        )

        # compute logprob for each candidate genre
        scores = {}
        for g in genres_eval:
            scores[g] = compute_genre_logprob(model, prompt, g)

        # pick max
        pred_genre = max(scores.items(), key=lambda x: x[1])[0]

        if pred_genre == true_g:
            correct += 1
        total += 1

    for h in handles:
        h.remove()

    acc = correct / total if total > 0 else 0.0
    print(f"{genre_name:15s} | mode={mode:8s} | acc={acc:.3f} | n={total}")
    return acc


# ======================
# 6. Run experiment for all genres
# ======================
results = []

for g in genres_eval:
    if not N[g]:
        print(f"Skipping {g}: no neurons recorded.")
        continue

    base_acc    = evaluate_genre_mode(g, mode="base")
    ablate_acc  = evaluate_genre_mode(g, mode="ablate")
    amplify_acc = evaluate_genre_mode(g, mode="amplify", scale=AMPLIFY_SCALE)

    results.append({
        "genre": g,
        "base": base_acc,
        "ablate": ablate_acc,
        "amplify": amplify_acc,
    })

# ======================
# 7. Print and plot
# ======================
import pandas as pd

if not results:
    print("No results")
else:
    df = pd.DataFrame(results)
    print(df)

    # Save table for report
    df.to_csv("Base_Ablate_Amplify.csv", index=False)

    # Plot
    x = np.arange(len(df))
    width = 0.25

    plt.figure(figsize=(12, 4))
    plt.bar(x - width, df["base"],    width=width, label="Base")
    plt.bar(x,         df["ablate"],  width=width, label="Ablate")
    plt.bar(x + width, df["amplify"], width=width, label="Amplify")
    plt.xticks(x, df["genre"])
    plt.ylabel("Accuracy")
    plt.title("Genre Prediction Accuracy: Base vs Ablate vs Amplify (QRNCA Neurons)")
    plt.legend()
    plt.tight_layout()
    plt.savefig("Base_Ablate_Amplify.png")



Loaded neurons for 18 genres.
Loaded 18 test samples.
Genres in test data: ["Children's", 'Comedy', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'Western', 'IMAX', 'Other', 'Animation', 'Adventure', 'Crime']
Evaluating on genres: ["Children's", 'Comedy', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'Western', 'IMAX', 'Other', 'Animation', 'Adventure', 'Crime']


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Model loaded.


In [ ]:

"""
neuron_contrib_six.py
---------------------
Compute per-neuron contribution scores for six genres:
Action, Comedy, Drama, Horror, Romance, Sci-Fi.

Contribution score for a neuron (layer, idx) on genre g is:
  mean_over_samples( log P_g(base) - log P_g(ablated_single_neuron) )
where P_g is the label probability computed via next-token log-prob
on a multiple-choice prompt.

Inputs (must exist):
- ./data/movie_sgcate/neurons_by_genre.json        # QRNCA neurons per genre
- ./data/movie_sgcate/genre_test_all_genres_hard.json  # test set with {"text","genre"}

Output:
- neuron_contrib_six.csv
- neuron_contrib_six_raw.json   (full mapping)
- optional plots: top_neurons_<GENRE>.png

Run:
  python neuron_contrib_six.py
"""

import os, json
from collections import defaultdict
from typing import List, Tuple, Dict

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# CONFIG (edit as needed)
# -----------------------------
NEURON_JSON_PATH = "./data/movie_sgcate/neurons_by_genre.json"
TEST_JSON_PATH   = "./data/movie_sgcate/genre_test_all_genres_hard.json"
BASE_MODEL       = "meta-llama/Meta-Llama-3-8B-Instruct"
DEVICE           = "cuda" if torch.cuda.is_available() else "cpu"

TARGET_GENRES = ["Action", "Comedy", "Drama", "Horror", "Romance", "Sci-Fi"]

MAX_SAMPLES_PER_GENRE = 16     # increase for more stable estimates
MAX_NEURONS_PER_GENRE = 150    # cap for speed; set higher if you have GPU time
TOP_K_EXPORT           = 20     # how many top neurons to keep per genre

# -----------------------------
# Load files
# -----------------------------
if not os.path.exists(NEURON_JSON_PATH):
    raise FileNotFoundError(f"Missing {NEURON_JSON_PATH}")
if not os.path.exists(TEST_JSON_PATH):
    raise FileNotFoundError(f"Missing {TEST_JSON_PATH}")

with open(NEURON_JSON_PATH, "r") as f:
    neurons_by_genre = json.load(f)
N: Dict[str, set] = {g: set(tuple(x) for x in v) for g, v in neurons_by_genre.items()}

with open(TEST_JSON_PATH, "r") as f:
    raw = json.load(f)

samples = []
for ex in raw:
    t = ex.get("text"); g = ex.get("genre")
    if isinstance(t, str) and isinstance(g, str):
        samples.append({"text": t, "genre": g})

samples_by_genre = defaultdict(list)
for ex in samples:
    samples_by_genre[ex["genre"]].append(ex)

# Filter to the six target genres present in both files
genres_eval = [g for g in TARGET_GENRES if g in N and g in samples_by_genre and len(samples_by_genre[g]) > 0]
if not genres_eval:
    raise ValueError("No overlap between target genres, neuron file, and test file.")
print("Evaluating genres:", genres_eval)

# -----------------------------
# Load model
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    device_map="auto" if DEVICE == "cuda" else None,
)
model.eval()

# -----------------------------
# Helpers
# -----------------------------
def safe_single_neuron_hook(layer_id: int, neuron_id: int):
    """
    Forward hook that zeros a SINGLE neuron in a given layer.
    Handles tuple outputs and invalid indices.
    """
    target = model.model.layers[layer_id]
    def hook_fn(module, inputs, output):
        hidden = output[0] if isinstance(output, tuple) else output
        hidden_dim = hidden.size(-1)
        if not (0 <= neuron_id < hidden_dim):
            return output
        idx = torch.tensor([neuron_id], device=hidden.device)
        hidden[..., idx] = 0
        return (hidden,) + output[1:] if isinstance(output, tuple) else hidden
    return target.register_forward_hook(hook_fn)


def compute_label_logprob(prompt: str, label: str) -> float:
    """
    Compute log P(label | prompt) by summing next-token logprobs of label tokens.
    """
    full = prompt + label
    enc_full = tokenizer(full, return_tensors="pt").to(model.device)
    full_ids = enc_full["input_ids"][0]

    enc_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
    prompt_ids = enc_prompt["input_ids"][0]

    len_prompt = prompt_ids.size(0)
    label_ids = full_ids[len_prompt:]

    with torch.no_grad():
        out = model(full_ids.unsqueeze(0))
        logits = out.logits[0]
    logprobs = torch.log_softmax(logits, dim=-1)

    total = 0.0
    for i, tok in enumerate(label_ids):
        pos = len_prompt + i - 1
        if 0 <= pos < logprobs.size(0):
            total += logprobs[pos, tok].item()
    return total


def build_prompt(text: str, candidate_genres: List[str]) -> str:
    return (
        "Read the following movie description and choose the most appropriate genre "
        "from the candidate list.\n\n"
        f"Description: {text}\n\n"
        f"Candidate genres: {', '.join(candidate_genres)}\n"
        "Answer with only one genre: "
    )


def neuron_contrib_for_genre(genre: str, neurons: List[Tuple[int,int]], candidates: List[str]) -> Dict[Tuple[int,int], float]:
    """
    For a given genre, compute contribution score for each neuron:
      mean_k( logP_g(base) - logP_g(ablated_neuron) )
    """
    # use subset of samples
    genre_samples = samples_by_genre[genre][:MAX_SAMPLES_PER_GENRE]
    if not genre_samples:
        return {}

    # thin neuron list for speed
    pruned = list(neurons)[:MAX_NEURONS_PER_GENRE]

    # Precompute base logP for each sample
    prompts = []
    base_logps = []
    for ex in genre_samples:
        p = build_prompt(ex["text"], candidates)
        prompts.append(p)
        base_logps.append(compute_label_logprob(p, genre))

    contrib = {}
    for (layer, idx) in pruned:
        handle = safe_single_neuron_hook(layer, idx)
        deltas = []
        try:
            for k, p in enumerate(prompts):
                lp_abl = compute_label_logprob(p, genre)
                deltas.append(base_logps[k] - lp_abl)  # >0 means this neuron helps
        finally:
            handle.remove()
        contrib[(layer, idx)] = float(np.mean(deltas)) if deltas else 0.0

    return contrib

# -----------------------------
# Run for all six genres and save
# -----------------------------
all_results = {}
rows = []

for g in genres_eval:
    neurons_g = list(N[g])
    if not neurons_g:
        print(f"[skip] {g}: no neurons")
        continue
    print(f"[genre] {g}: scoring up to {min(len(neurons_g), MAX_NEURONS_PER_GENRE)} neurons...")
    scores = neuron_contrib_for_genre(g, neurons_g, genres_eval)
    all_results[g] = scores

    # top-K rows for compact CSV
    top_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:TOP_K_EXPORT]
    for (layer, idx), s in top_items:
        rows.append({"genre": g, "layer": layer, "neuron": idx, "contribution": s})

# Save outputs
pd.DataFrame(rows).to_csv("neuron_contrib.csv", index=False)
with open("neuron_contrib.json", "w") as f:
    json.dump({g: {f"{k[0]}:{k[1]}": v for k, v in d.items()} for g, d in all_results.items()}, f, indent=2)
